In [1]:
import numpy as np
import os
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from keras.models import load_model
from keras.models import Sequential
from sklearn.utils import shuffle
from keras.layers import Dropout
from keras.layers import Dense
from sklearn import preprocessing
%matplotlib inline
import pandas as pd

from utils import *

Using TensorFlow backend.


In [2]:
folder_name="1_all_NNs"

### GET DATA

In [3]:
# read data
train_df = pd.read_csv('data/ADA_girl_boy/data41ADA_girl.csv')
test_df = pd.read_csv('data/ADA_girl_boy/data41ADA_boy.csv')

### PREPROCESS DATA

#### Test and train split

In [4]:
#     """
#     get data and return preprocessed data
    
#     Parameters
#     ----------
#     train_df: train data
#     test_df: test data
#     label_increment: increment model index only for one vs. all models, default False
#     categorical: categorical preperation of data, default True
#     category_size: category size for catageorical preperation, default: 41
#     normalize: normalize data, default True;
    
#     Return
#     ------
#     X_out, y_out
#     """

In [ ]:
# type: no_category, yes_incement
X_train, y_train = utils_prepare_data(train_df, label_increment=True, categorical=False)
X_test, y_test = utils_prepare_data(test_df, label_increment=True, categorical=False)

In [ ]:
print('Size of training matrix:', X_train.shape)
print('Size of testing matrix:', X_test.shape)

In [ ]:
# get unique labesl
unique_words = set(y_train)
print(unique_words)

### IMBALANCED DATASET
In this document we train our model by utilizing one vs all apprach. Thefore the data of single label (ones) is considerably low than that of others. Before train our model, we need to fix the imbalance dataset.

In [ ]:
#draw chart
def pie_chart(y,label):
    one_shape = y[y==label].shape[0]
    others_shape = y[y!=label].shape[0]
    plt.pie(
        list([one_shape, others_shape]),
        labels=['Label {}: {}'.format(label,one_shape),'{}: {}'.format('others',others_shape)]
    );

In [ ]:
# visualize imbalanced dataset: exmaple for label 1
pie_chart(y_train,1)

#### SMOTE
SMOTE (Synthetic Minority Over-Sampling Technique) is an over-sampling technique that introduces small perturbations to synthetic examples along the direction of existing samples to reduce overfitting. See original paper for detailed explanation of SMOTE.


#### SMOTE Implementation

There is a SMOTE implementation in imblearn package for scikit-learn. However, there is not an option to apply SMOTE with arbitrary percentages (SMOTE-100, SMOTE-300, etc.); it simply balances all the classes. And also since SMOTE is not a hard to implement algorithm, we provide our own implementation.

In [ ]:
from sklearn.neighbors import NearestNeighbors
from random import randint


def smote(samples, amount, k=5):
    """
    Apply SMOTE algorithm to samples and return a new samples
    array with synthetically created samples.
    
    Parameters
    ----------
    samples: (n_samples, n_features) samples array to be sent to
             SMOTE algorithm.
    amount: Percentage of newly created synthetic samples. (E.g.
            amount=100 would create as many synthetic examples
            as existing ones).
    k: Number of nearest neighbors in SMOTE algorithm.
    
    Returns
    -------
    out: ((1 + amount/100)*n_samples, n_features) samples array containing
         the original and the newly created synthetic examples.
         
    References
    ----------
    http://www.jair.org/media/953/live-953-2037-jair.pdf
    """
    samples = np.copy(samples)
    n_samples, n_features = samples.shape
    # handle amount < 100 case
    if amount < 100:
        num_samples = int(len(samples)*(amount/100))
        np.shuffle(samples)
        samples = samples[:num_samples, :]
        amount = 100
    amount = int(amount/100)
    synthetic = np.empty((n_samples*amount, n_features))
    # find k nearest neighbors of each point and store it in nnarray
    nbrs = NearestNeighbors(n_neighbors=k + 1).fit(samples)
    _, nnarray = nbrs.kneighbors(samples)
    nnarray = nnarray[:, 1:]  # get rid of self-nearest-neighbor.
    # create synthetic examples and store them in synthetic.
    for i, neighbors in enumerate(nnarray):
        for j in range(amount):
            chosen = neighbors[randint(0, k - 1)]
            diff = samples[chosen] - samples[i]
            gap = np.random.rand(n_features)
            synthetic[i*amount + j] = samples[i] + gap*diff
    out = np.vstack((samples, synthetic))
    
    return out

#### Visualizing Synthetic SMOTE Examples

In [ ]:
# generate sample array
arr1 = np.random.normal(loc=5, scale=2.5, size=(50, 2))
arr2 = np.random.normal(loc=0, scale=2.5, size=(20, 2))

In [ ]:
arr_smote = smote(arr2, 100)
plt.scatter(*arr1.T, label='Majority')
plt.scatter(*arr2.T, label='Minority')
plt.scatter(*arr_smote[25:, :].T, label='new SMOTE samples')
plt.legend();

In [ ]:
def smote_sampling(X, y, label):

    other_n = y[y!=label].shape[0]
    one_n = y[y==label].shape[0]
    smote_amount = int(((other_n-one_n)*100)/one_n)

    X_one = X[y==label]
    synthetic_data = smote(X_one, smote_amount)
    n_synthetic = len(synthetic_data)

    # merge synthetic examples with original examples
    X_out = np.vstack((X[y!=label], synthetic_data))
    y_out = np.concatenate((y[y!=label], [label]*n_synthetic))

    return X_out, y_out

In [ ]:
# example for label 1 after upsampling
X_out, y_out = smote_sampling(X_train, y_train, 1)
pie_chart(y_out,1)

In [ ]:
def ronadom_undersample(X,y,label):
    X1 = X[y==label]
    y1 = y[y==label]

    # define others labels
    # and reduce theri elements randomly
    X2 = X[y!=label]
    y2 = y[y!=label]
    idx = np.random.choice(np.arange(X2.shape[0]), int(X1.shape[0]*4), replace=False)
    X2_new = X2[idx,:]
    y2_new = y2[idx]

    X_out = np.vstack((X1, X2_new))
    y_out = np.concatenate((y1, y2_new))

    return X_out, y_out

In [ ]:
# example for label 1 after undersampling
X_out, y_out = ronadom_undersample(X_train, y_train, 1)
pie_chart(y_out,1)

In [ ]:
def multiply_oversmaple(X,y,label):
    
    X1 = X[y==label]
    y1 = y[y==label]

    # define others labels
    # and reduce theri elements randomly
    X2 = X[y!=label]
    y2 = y[y!=label]
    
    X1_new=[X1]*int(X2.shape[0]/X1.shape[0])
    X1_new=np.array(X1_new)
    X1_new=np.resize(X1_new,(X1_new.shape[0]*X1_new.shape[1],X2.shape[1]))
    
    y1_new=[y1]*int(y2.shape[0]/y1.shape[0])
    y1_new=np.array(y1_new)
    y1_new=np.resize(y1_new,(y1_new.shape[0]*y1_new.shape[1]))
                     
                     
    X_out = np.vstack((X1_new, X2))
    y_out = np.concatenate((y1_new, y2))

    return X_out, y_out

### TRAINER CLASS

In [ ]:
# class to handle NN processing
class NNTrainer(object):  
    '''
    Parameters
    ----------
    
    layer_activation: activation function for input and hidden layers
    covariance_type: activation function of end layer
    input_n_cols: numver of columns of input layer
    optimizer_function: optimization function
    loss_function: loss functions
    metrics_v: metric for evaluation result
    epochs_n: number of epoches to update train weights
    batch_size_n: batch size of fitted data
    validation_split_n: ratio of validation split in traning 
    
    choice of parameters depends on the data. 
    '''
    
    def __init__(self, layer_activation='relu', end_layer_activation='softmax',input_n_cols=2808,
                 optimizer_function='adam',loss_function='categorical_crossentropy',metrics_v='accuracy',
                epochs_n=20, batch_size_n=20, validation_split_n=0.2):
        
        # initialize variables
        self.layer_activation = layer_activation
        self.end_layer_activation = end_layer_activation
        self.input_n_cols = input_n_cols
        self.optimizer_function = optimizer_function
        self.loss_function = loss_function
        self.metrics_v = metrics_v
        self.epochs_n = epochs_n
        self.batch_size_n =batch_size_n
        self.validation_split_n = validation_split_n
        
        
        # define model
        self.model = Sequential()
        #add layers to model and initialize
        self.model.add(Dense(200, activation=self.layer_activation, input_shape=(self.input_n_cols,)))
        self.model.add(Dense(200, activation=self.layer_activation, input_shape=(self.input_n_cols,)))
        self.model.add(Dense(200, activation=self.layer_activation))
#         self.model.add(Dropout(0.2))
#         self.model.add(Dense(50, activation=self.layer_activation))
        self.model.add(Dense(2, activation=self.end_layer_activation))
        
        # compile model
        self.model.compile(optimizer=self.optimizer_function, 
                           loss=self.loss_function,metrics=[self.metrics_v])
            
    #train mode
    def train(self, X_train, y_train):
        # ingonre divisin by 0
        # np.seterr(all='ignore') 
        #train model
        self.model.fit(X_train, y_train, epochs=self.epochs_n,
                       batch_size=self.batch_size_n,validation_split=self.validation_split_n)
         
    # run the model on new data and get score
    def predict_probability(self, X_test):
        scores = self.model.predict_proba(X_test)
        return scores

    
    # return model
    def model_evaluate(self, X_test, y_test):
        scores = self.model.evaluate(X_test, y_test, verbose=0)
        return scores[1]*100
    
    # return model
    def get_nn_model():
        return self.model
    
    def save_modle(self, folder_name, model_name):
        # Creates a HDF5 file 'my_model.h5'
        self.model.save('models/{path}/{model}.h5'.format(path=folder_name,model=model_name))

### TRAIN MODELS

Train each model seperately. 

In [ ]:
# X_t_data=[]
# y_t_data=[]

# for label in unique_words:
    
#     # random undersampling 
#     # X_out, y_out = ronadom_undersample(X_train, y_train, int(label))
    
#     # oversmapling smote
#     # X_out,y_out = smote_sampling(X_train, y_train, int(label))
    
#     print(label)
    
#     # oversampling multiply
#     X_out, y_out = multiply_oversmaple(X_train, y_train, int(label))
    
#     # shullfe data
    
#     time.sleep(30)
    
#     print("shuffle")
    
#     X_out, y_out = shuffle(X_out, y_out, random_state=0)
    
#     X_t_data.insert(int(label)-1,X_out)
#     y_t_data.insert(int(label)-1,y_out)

In [ ]:
nn_models = [] # list to keep all models
input_n_cols=X_train.shape[1]

### for each model
for label in unique_words:
    
#     X_out=X_t_data[int(label)-1]
#     y_out=y_t_data[int(label)-1]

    

    # generate traget lables make others zero: one vs all
    target = np.zeros((len(y_train),2),dtype=int)
    for i,l_i in enumerate(y_train):
        if(int(l_i)==int(label)):
            target[i][0]=1
        else: 
            target[i][1]=1
    
    print(int(label))
        
    # train model
    nn_trainer = NNTrainer(input_n_cols=input_n_cols)
    nn_trainer.train(X_train, target)
    
    nn_trainer.save_modle(folder_name, "ada_girls_model_{}".format(int(label)))
    
    # append model
    nn_models.append((nn_trainer, label))
    nn_trainer = None

### EVALUATE MODELS

Given each test data, we run all the models on it data and pick the one with the best score.

In [ ]:
# get the model with best score

# as undersample data is marginal reduce the number of test data

logprob_1 = np.array([m[0].predict_probability(X_test)[:,0] for m in nn_models]) ## prob of one gorup
logprob_2 = np.array([m[0].predict_probability(X_test)[:,1] for m in nn_models]) ## prob of others group

In [ ]:
# predicted_label = np.argmax(logprob_1, axis=0)
# error = (predicted_label != y_test)
# print('Overall test accuracy: %.2f percent' % (100 * (1-np.mean(error))))

In [ ]:
logprob_1=logprob_1.T
logprob_2=logprob_2.T

In [ ]:
'''
how to evaluate evaluate model:
    select the ones with larget prob.
    if prob of ones are euqal for any model then select the model with min others prob
'''
 
result=np.zeros((y_test.shape[0]), dtype=int)

for i in range(y_test.shape[0]):
    max_array=[]
    max_n=-100
    idx=1
    max_min=100
    for j in range(41):
        max_array.append(logprob_1[i][j])
        
        if logprob_1[i][j]>max_n and logprob_1[i][j]>=.9:
            max_n=logprob_1[i][j]
            max_min=logprob_2[i][j]
            idx=(j+1)   
        if logprob_1[i][j]==max_n and logprob_2[i][j]<max_min:
            max_n=logprob_1[i][j]
            max_min=logprob_2[i][j]
            idx=(j+1)
            
            
    
    #sort max array
    max_array.sort()
    
    # compare result with the actual labels
    if(int(y_test[i])==int(idx) and max_array[-1]-max_array[-2]>=0.5):
        result[i]=1
        
        
overall_acc = np.mean(result)*100
print('Overall unseen test accuracy: %.2f percent' % overall_acc)

**Accuracy on unseen test data (one vs. all)**

* Normal Accuracy: 99.32

* Accuracy where (h1>=0.9): 98.57

* Accuracy where (h1>=0.9 and h1-h2>=0.5): 98.07

**Accuracy on unseen test ada girls data (ada girl model)**

* Normal Accuracy: 95.31

* Accuracy where (h1>=0.9): 91.66

* Accuracy where (h1>=0.9 and h1-h2>=0.5): 90.63

#### CHECK GENERALIZATION ON DIFFERENT MODELS

In [ ]:
'''
in order to check generalization of model, we'll test data 40(40th word features) on model 1 (1 word's data)
'''
# random undersampling 
# X_out, y_out = ronadom_undersample(X_train, y_train, int(1))

# oversmapling smote
# X_out, y_out = smote_sampling(X_train, y_train, int(1))

# oversampling multiply
X_out, y_out = multiply_oversmaple(X_train, y_train, int(1))

# shullfe data
X_out, y_out = shuffle(X_out, y_out, random_state=0)

target = np.zeros((len(y_out),2),dtype=int)

for i,l_i in enumerate(y_out):
    if(int(l_i)==40):
        target[i][0]=1
    else:         
        target[i][1]=0

In [ ]:
model_1_result_41=nn_models[0][0].model_evaluate(X_out,target)
print("%.2f%%" % (model_1_result_41))

As it seems not-targeted data on target model outputs a result with a marginal degree of certainty, 49.03%. This indicates that models trained can make good generalization.